<a href="https://colab.research.google.com/github/seanmcalevey/side_project/blob/master/gru_e512_g256_g256_d1024_(30steps).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ! pip install -U tensorflow

In [ ]:
 import numpy as np
import pandas as pd
import re
import tensorflow as tf
from tensorflow import keras

In [ ]:
tf.__version__

'2.1.0'

In [ ]:
from google.colab import drive

drive.mount('/content/drive/', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [ ]:
master_df = pd.read_csv('/content/drive/My Drive/Consumer_Complaints.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5,6,11,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Sample and Process Text

In [ ]:
proc_df = master_df.dropna(subset=['Consumer complaint narrative']).sample(20000, random_state=55)

clean = [re.sub('[^A-Za-z.,\s\']', '', nar) for nar in proc_df['Consumer complaint narrative']]

split_word_nars = [nar.split() for nar in clean]

"""Contractions Import"""

import sys
sys.path.append('/content/drive/My Drive')
from english_contractions import replace_contraction

"""End Contraction Import"""

new_words = []

for nar in split_word_nars:

  nar_words = []

  for word in nar:

    if re.search('\w+[.]', word):

      splitted = word.split('.')

      tmp_words = replace_contraction(splitted[0].lower())

      for w in tmp_words.split():

        nar_words.append(w)

      nar_words.append('.')
    
    elif re.search('\w+[,]', word):
      
      splitted = word.split(',')

      tmp_words = replace_contraction(splitted[0].lower())

      for w in tmp_words.split():

        nar_words.append(w)

      nar_words.append(',')
    
    elif re.match('[.]', word):
      
      placeholder = 1
    
    else:

      tmp_words = replace_contraction(word)

      for w in tmp_words.split():

        nar_words.append(w)

  new_words.append(nar_words)

words = [word for words in new_words for word in words]

## Tokenize Text

In [ ]:
from keras.preprocessing.text import Tokenizer

num_words = 10000

tokenizer = Tokenizer(num_words=num_words, filters='')

tokenizer.fit_on_texts(words)

encoded = tokenizer.texts_to_sequences(words)

max_id = num_words

dataset_size = tokenizer.document_count

flat_encoded = [enc for encoder in encoded for enc in encoder]

print(max_id)

Using TensorFlow backend.


10000


## Split Train and Val Sets

In [ ]:
train_size = dataset_size * 90 // 100

dataset = tf.data.Dataset.from_tensor_slices(flat_encoded[:train_size])

val_dataset = tf.data.Dataset.from_tensor_slices(flat_encoded[train_size:])

## Batch and Prepare Train Set

In [ ]:
batch_size = 256

n_steps = 30

window_length = n_steps + 1

dataset = dataset.window(window_length, shift=1, drop_remainder=True)

dataset = dataset.flat_map(lambda window: window.batch(window_length))

dataset = dataset.shuffle(10000).batch(batch_size)

dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

# dataset = dataset.map(lambda X_batch, y_batch: (tf.one_hot(X_batch, depth=max_id), y_batch))

dataset = dataset.prefetch(1)

## Batch and Prepare Val Set

In [ ]:
val_dataset = val_dataset.window(window_length, shift=1, drop_remainder=True)

val_dataset = val_dataset.flat_map(lambda window: window.batch(window_length))

val_dataset = val_dataset.shuffle(10000).batch(batch_size)

val_dataset = val_dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

# val_dataset = val_dataset.map(lambda X_batch, y_batch: (tf.one_hot(X_batch, depth=max_id), y_batch))

val_dataset = val_dataset.prefetch(1)

## Callbacks

In [ ]:
# Early Stopping

from keras.callbacks import EarlyStopping

early_stopper = EarlyStopping(monitor='val_loss', patience=1, restore_best_weights=True)

# Checkpoing Model Weights

import os

checkpoint_path = 'checkpoints/cp-{epoch:04d}.ckpt'

checkpoint_dir = os.path.dirname(checkpoint_path)

checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    period=1)

## Identify Last Epoch Checkpoint

In [ ]:
latest = tf.train.latest_checkpoint(checkpoint_dir)

latest

## Size of Train Set

In [ ]:
steps_per_epoch = train_size // batch_size

steps_per_epoch

14155

## Build and Train Model

In [ ]:
adam = tf.keras.optimizers.Adam()

model = tf.keras.models.Sequential([
                                    tf.keras.layers.Embedding(max_id, 512, input_shape=[None]),
                                    tf.keras.layers.GRU(256, return_sequences=True,
                                                        dropout=0.5, recurrent_dropout=0.5),
                                    tf.keras.layers.GRU(256, return_sequences=True,
                                                        dropout=0.5, recurrent_dropout=0.5),
                                    tf.keras.layers.Dropout(0.5),
                                    tf.keras.layers.Dense(1024, activation='sigmoid'),
                                    tf.keras.layers.Dense(max_id, activation='softmax')])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# model.load_weights(latest)

model.fit(dataset, epochs=2, validation_data=val_dataset, callbacks=[early_stopper, checkpoint_cb])

Epoch 1/2
  14156/Unknown - 3239s 229ms/step - loss: 3.9655 - accuracy: 0.2480
Epoch 00001: saving model to checkpoints/cp-0001.ckpt
14156/14156 [==============================] - 3347s 236ms/step - loss: 3.9655 - accuracy: 0.2480 - val_loss: 4.4203 - val_accuracy: 0.2411
Epoch 2/2
14155/14156 [============================>.] - ETA: 0s - loss: 3.7341 - accuracy: 0.2712
Epoch 00002: saving model to checkpoints/cp-0002.ckpt
14156/14156 [==============================] - 3326s 235ms/step - loss: 3.7340 - accuracy: 0.2712 - val_loss: 4.3053 - val_accuracy: 0.2488


In [ ]:
model.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_30 (GRU)                 (None, None, 256)         7878144   
_________________________________________________________________
gru_31 (GRU)                 (None, None, 256)         394752    
_________________________________________________________________
dropout_20 (Dropout)         (None, None, 256)         0         
_________________________________________________________________
dense_35 (Dense)             (None, None, 256)         65792     
_________________________________________________________________
dense_36 (Dense)             (None, None, 10000)       2570000   
Total params: 10,908,688
Trainable params: 10,908,688
Non-trainable params: 0
_________________________________________________________________


## Save the Trained Model

In [ ]:
drive_path = '/content/drive/'

file_path = 'My Drive/saved_keras_rnns/'

name = 'gru_1002.h5'

model.save(drive_path + file_path + name)

## Save the Tokenizer

In [ ]:
import pickle

tokenizer_name = 'tok_gru_1002.pkl'

pickle.dump(tokenizer, open(drive_path + file_path + tokenizer_name, 'wb'))

## Load a Trained Model

In [ ]:
name = 'gru_30_steps.h5'

new_model = keras.models.load_model(drive_path + file_path + name)

new_model.summary()

## Text Generation

In [ ]:
def text_clean(text):

  splits = text.split(' . ')[:-1]

  new_splits = []

  for split in splits[1:]:

    word_splits = split.split(' ')
    
    word_splits[0] = word_splits[0].capitalize()
    
    word_splits[-1] = ''.join([word_splits[-1], '.'])

    joined = ' '.join(word_splits)

    new_splits.append(joined)
  
  join_split = ' '.join(new_splits)

  return join_split

def preprocessor(text):

  # X = tokenizer.texts_to_sequences(text)

  # return tf.one_hot(X, max_id) # for no embedding

  # return tokenizer.texts_to_sequences(text) # for embedding

def next_word(text, model, temperature=1):
  
  X_new = preprocessor([text])

  y_proba = model.predict(X_new)[0, -1:, :]

  rescaled_logits = tf.math.log(y_proba) / temperature

  word_id = tf.random.categorical(rescaled_logits, num_samples=1)

  return tokenizer.sequences_to_texts(word_id.numpy())[0]

def complete_text(text, model, n_words=50, temperature=0.5):

  for _ in range(n_words):

    space = [' ', next_word(text, model, temperature)]
    
    text += ''.join(space)

  return text

In [ ]:
key_words = ['fcra', 'loan', 'bank', 'equifax breach', 'credit card', 'credit report']

temps = [0.15, 0.2, 0.25]

count = 0

for key_word in key_words:
  for temp in temps:
    count += 1
    print(f'{count} (kw: "{key_word}", tmp: {temp})', text_clean(complete_text(key_word, new_model_2, temperature=temp)))

1 (kw: "fcra", tmp: 0.15) I have been disputing this debt with transunion and i have never received any response from the provider of service. I have been disputing the pmi with the credit bureaus. I have been disputing this account with the credit bureaus.
2 (kw: "fcra", tmp: 0.2) I have been disputing this account with bsi financial services. I have been disputing this account with my credit report. I have been disputing this debt with the credit bureaus. I have been disputing this debt from my credit report.
3 (kw: "fcra", tmp: 0.25) I have contacted them about the process of the phone with the company and that my credit score is now. I have been trying to obtain a home and they have not been able to do so. I am not sure what happened to the.
4 (kw: "loan", tmp: 0.15) I was told that the check was rejected by seterus,. Inc. I was told that the check had bounced. I was told that the check was rejected by the bank of america.
5 (kw: "loan", tmp: 0.2) I have been a victim of identity t